In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("capstone") \
      .getOrCreate() 

In [3]:
rawdf = spark.read.csv("Capstone market analysis (4).csv",header = True)
rawdf.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [4]:
rawdf.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [5]:
 rawdf.createTempView("tempbank")

In [6]:
success = spark.sql("select (subscribed/total)*100 as marketing_success_rate from (select count(*) as subscribed from tempbank where y = 'yes') , (select count (*) as total from tempbank)").show()

+----------------------+
|marketing_success_rate|
+----------------------+
|    11.698480458295547|
+----------------------+



In [7]:
failure = spark.sql("select (not_subscribed/total)*100 as marketing_failure_rate from (select count(*) as not_subscribed from tempbank where y = 'no') , (select count (*) as total from tempbank)").show()        

+----------------------+
|marketing_failure_rate|
+----------------------+
|     88.30151954170445|
+----------------------+



In [8]:
max_age=spark.sql("select max(age) from tempbank").show()
min_age=spark.sql("select min(age) from tempbank").show()
average_age=spark.sql("select avg(age) from tempbank").show()


+--------+
|max(age)|
+--------+
|      95|
+--------+

+--------+
|min(age)|
+--------+
|      18|
+--------+

+-----------------+
|         avg(age)|
+-----------------+
|40.93621021432837|
+-----------------+



In [9]:
average_balance=spark.sql("select avg(balance) from tempbank").show()

+------------------+
|      avg(balance)|
+------------------+
|1362.2720576850766|
+------------------+



In [10]:
median = spark.sql("select percentile_approx(balance, 0.5) from tempbank").show()

+--------------------------------------+
|percentile_approx(balance, 0.5, 10000)|
+--------------------------------------+
|                                 448.0|
+--------------------------------------+



In [39]:
age = spark.sql("select age, count(*) as number from tempbank where y='yes' group by age ")
age.show()
age.createOrReplaceTempView("temp9")

+---+------+
|age|number|
+---+------+
| 51|    77|
| 54|    84|
| 29|   171|
| 69|    17|
| 42|   111|
| 73|    24|
| 87|     3|
| 64|    35|
| 30|   217|
| 34|   198|
| 59|    88|
| 28|   162|
| 22|    40|
| 85|     4|
| 35|   209|
| 52|    85|
| 71|    25|
| 47|   113|
| 43|   103|
| 31|   206|
+---+------+
only showing top 20 rows



In [37]:
age_total = spark.sql("select age, count(*) as number from tempbank where group by age ")
age_total.show()
age_total.createOrReplaceTempView("temp8")

+---+------+
|age|number|
+---+------+
| 51|   936|
| 54|   811|
| 29|  1185|
| 69|    44|
| 42|  1242|
| 73|    44|
| 87|     4|
| 64|    74|
| 30|  1757|
| 34|  1930|
| 59|   770|
| 28|  1038|
| 22|   129|
| 85|     5|
| 35|  1894|
| 52|   911|
| 71|    54|
| 47|  1088|
| 43|  1161|
| 31|  1996|
+---+------+
only showing top 20 rows



In [38]:
df11 = spark.sql("select a.age,a.number/b.number from temp9 a left join temp8 b  on a.age = b.age")
df11.show()

+---+-------------------+
|age|  (number / number)|
+---+-------------------+
| 51|0.08226495726495726|
| 54|0.10357583230579531|
| 29|0.14430379746835442|
| 69|0.38636363636363635|
| 42| 0.0893719806763285|
| 73| 0.5454545454545454|
| 87|               0.75|
| 64|0.47297297297297297|
| 30|0.12350597609561753|
| 34|0.10259067357512953|
| 59|0.11428571428571428|
| 28|0.15606936416184972|
| 22|0.31007751937984496|
| 85|                0.8|
| 35|0.11034846884899684|
| 52|0.09330406147091108|
| 71|0.46296296296296297|
| 47|0.10386029411764706|
| 43|0.08871662360034453|
| 31|0.10320641282565131|
+---+-------------------+
only showing top 20 rows



In [30]:
marital = spark.sql("select marital, count(*) as number from tempbank where y='yes' group by marital order by number desc")
marital.show()
marital.createOrReplaceTempView("temp7")

+--------+------+
| marital|number|
+--------+------+
| married|  2755|
|  single|  1912|
|divorced|   622|
+--------+------+



In [31]:
marital_total = spark.sql("select marital, count(*) as number from tempbank group by marital order by number desc")
marital_total.show()
marital_total.createOrReplaceTempView("temp6")

+--------+------+
| marital|number|
+--------+------+
| married| 27214|
|  single| 12790|
|divorced|  5207|
+--------+------+



In [32]:
df10 = spark.sql("select a.marital,a.number/b.number from temp7 a left join temp6 b  on a.marital = b.marital")
df10.show()

+--------+-------------------+
| marital|  (number / number)|
+--------+-------------------+
| married|0.10123465863158668|
|  single| 0.1494917904612979|
|divorced|0.11945458037257538|
+--------+-------------------+



In [13]:
 age_marital = spark.sql("select age,marital, count(*) as number from tempbank where y='yes' group by age,marital order by number desc").show()


+---+-------+------+
|age|marital|number|
+---+-------+------+
| 30| single|   151|
| 28| single|   138|
| 29| single|   133|
| 32| single|   124|
| 26| single|   121|
| 34|married|   118|
| 31| single|   111|
| 27| single|   110|
| 35|married|   101|
| 36|married|   100|
| 25| single|    99|
| 37|married|    98|
| 33|married|    97|
| 33| single|    97|
| 32|married|    87|
| 39|married|    87|
| 38|married|    86|
| 35| single|    84|
| 47|married|    83|
| 46|married|    80|
+---+-------+------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit


In [15]:
newdf=rawdf.withColumn("Newage", \
   when((rawdf.age < 20), lit("Teenage peoples")) \
     .when((rawdf.age > 20) & (rawdf.age <= 32), lit("Youngsters")) \
     .when((rawdf.age > 33) & (rawdf.age <= 55), lit("Middle Aged peoples")) \
     .otherwise(lit("Older peoples")) \
  )

In [16]:
newdf.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+-------------------+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|             Newage|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+-------------------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|      Older peoples|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|Middle Aged peoples|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|      Older peoples|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92| 

In [17]:
newdf.createOrReplaceTempView("tempbank1")

In [19]:
age_target_yes = spark.sql("select Newage, count(*) as yes_number from tempbank1 where y='yes' group by Newage order by yes_number desc ")
age_target_yes.createTempView("temp3")


In [21]:
age_target_no = spark.sql("select Newage, count(*) as No_number from tempbank1 where y='no' group by Newage order by No_number desc ")
age_target_no.createOrReplaceTempView("temp4")

In [23]:
df8 = spark.sql("select a.*,b.No_number from temp3 a left join temp4 b  on a.Newage = b.Newage")
df8.show()

+-------------------+----------+---------+
|             Newage|yes_number|No_number|
+-------------------+----------+---------+
|Middle Aged peoples|      2601|    24627|
|         Youngsters|      1539|     9475|
|      Older peoples|      1131|     5791|
|    Teenage peoples|        18|       29|
+-------------------+----------+---------+



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
x=df8["agedf"]
y=df8["yes_number", "No_number"]



# plot the dataframe
plt.plot(x, y, kind="line", figsize=(9, 9))
 
# print bar graph
plt.show()